In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import cumtrapz
from scipy.signal import detrend
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

plt.style.use(['ggplot', 'presentation'])

from madgwick import *

In [6]:
df3 = pd.read_csv('51888_1020030004.csv', header=None, names=['t', 'x', 'y', 'z'])
acc3 = df3[['x', 'y', 'z']].values
print(1 / np.mean(np.diff(df3['t'])))

197.24258229898155


In [7]:
df6 = pd.read_csv('6011802_0000000000.csv', header=None, names=['t', 'x', 'y', 'z', 'gx', 'gy', 'gz', '_1', '_2'])
acc = df6[['x', 'y', 'z']].values
gyr = df6[['gx', 'gy', 'gz']].values * np.pi / 180
print(1 / np.mean(np.diff(df6['t'])))

98.41345241248855


In [23]:
((vel3_wlk[-1] - vel3_wlk[0]) / (t3_wlk[-1] - t3_wlk[0])) * (t3_wlk - t3_wlk[0]).reshape((-1, 1))

array([[-0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [-1.41778625e-11, -5.67114502e-11,  0.00000000e+00],
       [-2.83564011e-11, -1.13425605e-10,  0.00000000e+00],
       ...,
       [-5.95757212e-08, -2.38302885e-07,  0.00000000e+00],
       [-5.95901830e-08, -2.38360732e-07,  0.00000000e+00],
       [-5.96046448e-08, -2.38418579e-07,  0.00000000e+00]])

In [28]:
acc3_wlk = acc3[21342:25490]
t3_wlk = df3.loc[21342:25489, 't'].values

vel3_wlk = cumtrapz(acc3_wlk * 9.81, t3_wlk, axis=0, initial=0)

vel3_wlk = vel3_wlk - ((vel3_wlk[-1] - vel3_wlk[0]) / (t3_wlk[-1] - t3_wlk[0])) * (t3_wlk - t3_wlk[0]).reshape((-1, 1))

pos3_wlk = cumtrapz(vel3_wlk, t3_wlk, axis=0, initial=0)


pca = PCA()
pos3_wlk_trf = pca.fit_transform(pos3_wlk)
print(pca.components_)


f, ax = plt.subplots(3, figsize=(9, 7), constrained_layout=True, sharex=True)
ax[0].plot(acc3_wlk)
ax[1].plot(vel3_wlk)


lbl = ['x', 'y', 'z']
for i in range(3):
    ax[2].plot(pos3_wlk[:, i], color=f'C{i}', label=lbl[i])
    ax[2].plot(pos3_wlk_trf[:, i], '--', color=f'C{i}')


[[ 0.11400086  0.18255993  0.9765632 ]
 [ 0.93691304 -0.34671108 -0.04455763]
 [ 0.33045085  0.9200344  -0.21056814]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
ahrs = MadgwickAHRS(sample_period=1/100)
q = np.zeros((acc.shape[0], 4))
R = np.zeros((q.shape[0], 3, 3))

for i in range(acc.shape[0]):
    q[i] = ahrs.updateIMU(gyr[i], acc[i])
    R[i] = quat2matrix(q[i])
    
acc_c = (R @ acc.reshape((-1, 3, 1))).reshape((-1, 3))
gyr_c = (R @ gyr.reshape((-1, 3, 1))).reshape((-1, 3))

acc_c_wlk = acc_c[8200:9700]

vel_c_wlk = detrend(cumtrapz(acc_c_wlk * 9.81, dx=1/100, axis=0), axis=0)
pos_c_wlk = cumtrapz(vel_c_wlk, dx=1/100, axis=0)

plt.close('all')
f, ax = plt.subplots(3, figsize=(9, 7), constrained_layout=True, sharex=True)
ax[0].plot(acc[8200:9700]);
ax[1].plot(vel_c_wlk);
ax[2].plot(pos_c_wlk);